In [1]:
import sys; sys.path.append('../../') 

from parsers.parser import *
from utils.data_cleaner import *
from data_collection_config import *

In [ ]:
with ShopEmsParser() as parser:
    product_links = parser.collect_product_links(url_source_1)

pd.DataFrame(product_links, columns=['product_links']).to_csv(f'{RAW_DIRECTORY}{EMSSHOP_URLS_FILE}', index=False, encoding='utf-8')

In [2]:
product_links = pd.read_csv(f'{RAW_DIRECTORY}{EMSSHOP_URLS_FILE}').drop_duplicates()

In [ ]:
with ShopEmsParser() as parser:
    # Создаем списки для накопления данных
    all_goods_data = []
    all_specs_data = []
    
    # Парсим детальную информацию
    for url in product_links['product_links']:
        product_info, specs = parser.parse_single_product(url)
        
        if product_info:
            all_goods_data.append(product_info)
            all_specs_data.extend(specs)
    
    # Создаем DataFrame из накопленных данных
    goods_df, specs_df = parser.create_dataframes(all_goods_data, all_specs_data)
    
    # # Сохраняем в CSV
    # goods_df.to_csv('goods.csv', index=False, encoding='utf-8')
    # specs_df.to_csv('other_specifications.csv', index=False, encoding='utf-8')
    

In [ ]:
goods_df.to_csv(f'{RAW_DIRECTORY}{EMSSHOP_GOODS}', index=False, encoding='utf-8')
specs_df.to_csv(f'{RAW_DIRECTORY}{EMSSHOP_OTHER_DETAILS_FILE}', index=False, encoding='utf-8')

# Объединение данных

### Общая структура

In [2]:
Goods = pd.DataFrame(
    columns=[
        'id', 
        'marketplace', 
        'name', 
        'price',
        'rating', 
        'feedbacks', 
        'brand', 
        'description'
    ]
)

Other_characteristicks = pd.DataFrame(
    columns=[
        'id', 
        'marketplace', 
        'good_id',
        'name', 
        'value'
    ]
)

Feedbacks = pd.DataFrame(
    columns=[
        'id', 
        'good_id', 
        'rating',
        'advantage', 
        'disadvantage',
        'comment'
    ]
)

### Создание таблицы Goods

In [8]:
emsshop_goods = pd.read_csv(f'{RAW_DIRECTORY}{EMSSHOP_GOODS}')
noblerise_goods = pd.read_csv(f'{RAW_DIRECTORY}{NOBLERISE_GOODS}')

emsshop_goods.head(3)

,url,name,price,rating,feedbacks,brand,description
0,https://shopems.ru/ems-oborudovanie/xbody-newave,XBody Newave,990000.0,NaN,NaN,XBody,"XBody - Венгерская компания, основанная в 2010..."
1,https://shopems.ru/ems-oborudovanie/justfithome,Justfit Home,220000.0,NaN,NaN,Justfit,Justfit Home – идеальный EMS тренажер для дома...
2,https://shopems.ru/ems-oborudovanie/miha-bodytec,Miha Bodytec 2,1300000.0,NaN,NaN,Miha,EMS тренажер произведен компанией Miha Bodytec...


In [9]:
noblerise_goods.head(3)

,url,name,price,rating,feedbacks,brand,description
0,https://emsnoble.ru/catalog/for-home/,Комплект NobleRise «Домашний»,269000,NaN,NaN,NobleRise,EMS-костюмы созданные для многократного ежедне...


In [11]:
Goods = pd.concat([emsshop_goods, noblerise_goods], ignore_index=True)

# Goods['price'] = Goods['price'].astype(int)

Goods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   url          18 non-null     object 
 1   name         18 non-null     object 
 2   price        17 non-null     float64
 3   rating       0 non-null      float64
 4   feedbacks    0 non-null      float64
 5   brand        18 non-null     object 
 6   description  17 non-null     object 
dtypes: float64(3), object(4)
memory usage: 1.1+ KB


### Создание таблицы Other_characteristics

In [14]:
emsshop_other_characteristics = pd.read_csv(f'{RAW_DIRECTORY}{EMSSHOP_OTHER_DETAILS_FILE}')
noblerise_other_characteristics = pd.read_csv(f'{RAW_DIRECTORY}{NOBLERISE_OTHER_DETAILS_FILE}')


emsshop_other_characteristics

,url,name,value
0,https://shopems.ru/ems-oborudovanie/xbody-newave,Количество каналов,10
1,https://shopems.ru/ems-oborudovanie/xbody-newave,"Количество костюмов, необходимое для коммерчес...",3
2,https://shopems.ru/ems-oborudovanie/xbody-newave,Срок службы костюмов при большой загрузке,36 месяцев
3,https://shopems.ru/ems-oborudovanie/xbody-newave,Стоимость костюма,1000 евро
4,https://shopems.ru/ems-oborudovanie/xbody-newave,Съёмные электроды,✔
...,...,...,...
145,https://shopems.ru/ems-oborudovanie/visionbody,Система учета,✔
146,https://shopems.ru/ems-oborudovanie/visionbody,Страна производства,Германия
147,https://shopems.ru/ems-oborudovanie/visionbody,Срок гарантии,2 года
148,https://shopems.ru/ems-oborudovanie/visionbody,Оценка качества,4 качество


In [15]:
noblerise_other_characteristics

,url,name,value
0,https://emsnoble.ru/catalog/for-home/,Комплектация,EMS-костюм в сборе с проводкой и электродами —...
1,https://emsnoble.ru/catalog/for-home/,Комплектация,Генератор импульсов NobleRise — 1 шт.
2,https://emsnoble.ru/catalog/for-home/,Комплектация,Электроды с графитовым покрытием — 20 шт.
3,https://emsnoble.ru/catalog/for-home/,Комплектация,Доступ к ПО NobleRise EMS Trainer для некоммер...
4,https://emsnoble.ru/catalog/for-home/,Комплектация,Комплект нательного белья – 1 шт.
5,https://emsnoble.ru/catalog/for-home/,Комплектация,Зарядное устройство – 1 шт.
6,https://emsnoble.ru/catalog/for-home/,Комплектация,Инструкция на русском языке — 1 шт.
7,https://emsnoble.ru/catalog/for-home/,Комплектация,Пульверизатор — 1 шт.
8,https://emsnoble.ru/catalog/for-home/,Комплектация,Сертификат на оборудование NobleRise — 1 шт.


In [16]:
Other_characteristicks = pd.concat([emsshop_other_characteristics, noblerise_other_characteristics], ignore_index=True)

Other_characteristicks

,url,name,value
0,https://shopems.ru/ems-oborudovanie/xbody-newave,Количество каналов,10
1,https://shopems.ru/ems-oborudovanie/xbody-newave,"Количество костюмов, необходимое для коммерчес...",3
2,https://shopems.ru/ems-oborudovanie/xbody-newave,Срок службы костюмов при большой загрузке,36 месяцев
3,https://shopems.ru/ems-oborudovanie/xbody-newave,Стоимость костюма,1000 евро
4,https://shopems.ru/ems-oborudovanie/xbody-newave,Съёмные электроды,✔
...,...,...,...
154,https://emsnoble.ru/catalog/for-home/,Комплектация,Комплект нательного белья – 1 шт.
155,https://emsnoble.ru/catalog/for-home/,Комплектация,Зарядное устройство – 1 шт.
156,https://emsnoble.ru/catalog/for-home/,Комплектация,Инструкция на русском языке — 1 шт.
157,https://emsnoble.ru/catalog/for-home/,Комплектация,Пульверизатор — 1 шт.


### Запись всех данных

In [17]:
Goods.to_csv(f'{CLEAR_DIRECTORY}{CLEAR_GOODS_FILE}', index=False)
Other_characteristicks.to_csv(f'{CLEAR_DIRECTORY}{CLEAR_OTHER_CHARACTERISTICKS_FILE}', index=False)